# Sign up for News API (old Google news API)
* Import modules newsapi, pandas, json2html, boto3.
* Enter search keyword.
* Adjust from date and end time.
* Save results to .csv file.

In [2]:
from datetime import datetime, timezone, timedelta
from newsapi import NewsApiClient
import pandas as pd
from json2html import *
import boto3
from botocore.exceptions import ClientError

today = str(datetime)
newsapi = NewsApiClient(api_key='XXX')
search = 'XXx'

from_date = datetime.now(timezone.utc) + timedelta(days=-7)
end_time = datetime.now(timezone.utc) + timedelta(days=0)

all_articles = newsapi.get_everything(q=search,
                                      from_param=from_date,
                                      to=end_time,
                                      language='en',
                                      sort_by='relevancy'
                                      )
# Create Pandas DataFrame
df =pd.DataFrame(all_articles['articles'])
df.to_csv(str(datetime.now(timezone.utc)) + '-' + search + '-'+'news.csv')

# Convert json to htlm then send email notification.
* Setup AWS SES in order to send email notifications to recipients.

In [ ]:
# convert json to html
from datetime import date, datetime, timezone, timedelta
bodyconverted = json2html.convert(json=all_articles)

dt = date.today()
iso_date = dt.isoformat()

def send_email():
    SENDER = "XXX"  # must be verified in AWS SES Email
    RECIPIENT = "XXX"  # must be verified in AWS SES Email
    # RECIPIENT2 = "XXX" # must be verified in AWS SES Email

    # If necessary, replace us-west-2 with the AWS Region you're using for Amazon SES.
    AWS_REGION = "XXX"

    # The subject line for the email.
    SUBJECT = search + "News" + ":" + iso_date

    # The email body for recipients with non-HTML email clients.
    # BODY_TEXT = (itemstring)

    # The HTML body of the email.
    BODY_HTML = bodyconverted

    # The character encoding for the email.
    CHARSET = "UTF-8"

    # Create a new SES resource and specify a region.
    client = boto3.client('ses', region_name=AWS_REGION)

    # Try to send the email.
    try:
        # Provide the contents of the email.
        response = client.send_email(
            Destination={
                'ToAddresses': [
                    RECIPIENT
                   # RECIPIENT2
                ],
            },
            Message={
                'Body': {
                    'Html': {

                        'Data': BODY_HTML
                    },
                    # 'Text': {

                    #     'Data': BODY_TEXT
                    # },
                },
                'Subject': {

                    'Data': SUBJECT
                },
            },
            Source=SENDER
        )
    # Display an error if something goes wrong.
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        print("Email sent! Message ID:"),
        print(response['MessageId'])

# uncomment this lambda when moving it to AWS. Will run in container image in lambda

# def lambda_handler(event, context):
send_email()


# Get notifications on SNS (email or text)
* Setup SNS in AWS
* Setup Subscriptions on SNS Topic ARN

In [ ]:
import json
import logging

# Initialize logger and set log level
logger = logging.getLogger()
logger.setLevel(logging.INFO)

def send_sns():
   notification = all_articles
   client = boto3.client('sns')
   response = client.publish (
      TargetArn = "XXX",
      # PhoneNumber = 'XXX',
      Message = json.dumps(notification, indent=4, sort_keys=False),
      Subject = search + "News" + ":",
      MessageStructure = 'str'
   )
   logger.info(response)
   
   return {
      'statusCode': 200,
      'body': json.dumps(response)
   }
send_sns()